In [ ]:
import pandas as pd
import re
import os
from datasets import Dataset, Audio, DatasetDict
from pathlib import Path

from sklearn.model_selection import train_test_split

In [ ]:
from huggingface_hub import login
login()

In [ ]:
dataset = pd.read_json('/kaggle/input/output/output_cy_data.json')
full_audio_segments = '/kaggle/input/full-audio-segments/segments'

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
# Заменить префикс пути в столбце
df['audio_filepath'] = df['audio_filepath'].str.replace('./Audio_Files/', '/kaggle/input/audio-files/Audio_Files/', regex=False)

In [ ]:
df['audio_segments'] = dataset.apply(lambda row: 
    f"{full_audio_segments}/{Path(row['audio_filepath']).stem}_segment_{row.name:06d}.wav", 
    axis=1
)

In [ ]:
df = df.drop(columns=['audio_filepath'])
df = df[df['audio_segments'].apply(lambda x: Path(x).exists())]

In [ ]:
hf_dataset = Dataset.from_pandas(df)

In [ ]:
hf_dataset = hf_dataset.cast_column("audio_segments", Audio(sampling_rate=24000))

In [ ]:
hf_dataset = hf_dataset.rename_column("audio_segments", "audio")
hf_dataset = hf_dataset.rename_column("text", "sentence")

In [ ]:
hf_dataset

In [ ]:
# hf_dataset = hf_dataset.remove_columns(['__index_level_0__'])
hf_dataset = hf_dataset.cast_column("audio", Audio(sampling_rate=24000))

In [ ]:
###### SPLIT ########
train_test_split = hf_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
temp_dataset = train_test_split['test']

In [ ]:
val_test_split = temp_dataset.train_test_split(test_size=0.5, seed=42)
validation_dataset = val_test_split['train']
test_dataset = val_test_split['test'] 

In [ ]:
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})
########################

In [ ]:
### HuggingFace push
dataset_dict.push_to_hub("Elormiden/greek-speech-dataset")